In [ ]:

### Run this cell before continuing.
  
    

## Title

## Introduction

## Preliminary Results


    Demonstrate that the dataset can be read from the web into R.
    Clean and wrangle your data into a tidy format.
    Plot the relevant raw data, tailoring your plot in a way that addresses your question.
    Compute estimates of the parameter you identified across your groups. Present this in a table. If relevant, include these estimates in your plot.


## Methods: Plan


    What do you expect to find?
    What impact could such findings have?
    What future questions could this lead to?


##  References

At least two citations of literature relevant to the project. The citation format is your choice – just be consistent. Make sure to cite the source of your data as well.

- 
- 
...